# Multi Layer Perceptron 예제

이 예제를 통해 TensorFlow 2.0의 기본적인 사용법을 간단히 살펴봅니다.

먼저, TensorFlow를 import합니다.

In [ ]:
import tensorflow as tf

## 데이터 로드

Iris라는 간단한 데이터를 로드합니다.

이는 붓꽃에 대한 데이터로, 꽃받침의 길이, 꽃받침의 너비, 꽃잎의 길이, 꽃잎의 너비 총 4가지 필드를 가집니다.

그리고 class 수는 setosa / versicolor / virginica 로 총 3개입니다.

<br>

※ 시험 진행시 보통 데이터 로드 부분은 미리 코드로 주어집니다.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

x = iris['data']
y = iris['target']

# 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

## 로드한 데이터 확인

로드한 데이터의 형태를 확인해봅니다.

x_train은 (105, 4)의 shape를 가지는 numpy array입니다.

즉, 위에 언급한 4가지 필드를 가지는 데이터가 총 105개 있습니다.

y_train은 (105,)의 shape를 가지는 numpy array입니다.

실제 값을 보면, 0, 1, 2 중 하나의 label을 나타냄을 알 수 있습니다.

In [ ]:
print(type(x_train))
print(x_train.shape)
print(type(y_train))
print(y_train.shape)
print(y_train[:10])

## 네트워크 정의

tf.keras.models.Sequential을 이용하여 여러 layer를 원하는 대로 쌓아 네트워크를 구성할 수 있습니다.

아래의 model_1처럼 tf.keras.layers 아래에 있는 각종 layer들을 list로 구성하여 Sequential()을 호출하거나,

model_2처럼 add()를 이용해 차례차례 layer를 추가할 수 있습니다.

아래의 model_1과 model_2는 동일한 구조의 네트워크를 구성합니다.

이 네트워크에는 32 개의 node를 포함한 hidden layer가 하나 있고,

해당 layer에 20% 확률로 dropout을 적용하며

Class 수에 맞게 3개의 node를 가지는 output layer가 있습니다.

In [ ]:
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3, activation='softmax')
])

model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)))
model_2.add(tf.keras.layers.Dropout(0.2))
model_2.add(tf.keras.layers.Dense(3, activation='softmax'))

## 각 Layer에 대한 설명

위에서 사용한 각 layer들의 의미는 아래와 같습니다.

이 내용들은 TensorFlow 공식 document의

https://www.tensorflow.org/api_docs/python/tf/keras/layers

아래의 각 layer별 설명에서도 확인하실 수 있습니다.

**시험 응시 중에 TensorFlow 공식 document를 참고하는 것이 허용되므로 필요에 따라 활용하시면 좋습니다**

<br>

### Dense()

Fully Connected Layer 하나를 추가합니다.

첫번째로 받는 argument는 해당 layer 내의 node 수 입니다.

첫 번째 Dense layer는 32개의 node를 가지는 hidden layer이고,

마지막 Dense layer는 class 수에 맞게 3개의 node를 가지는 output layer입니다.

<br>

Activation function을 설정할 수 있습니다.

Hidden layer들에는 빠른 수렴을 위해 ReLU를 권장합니다.

이진 분류 문제의 output layer에는 sigmoid를 권장합니다.

Multi-class 분류 문제의 output layer에는 softmax를 권장합니다.

<br>

### Dropout()

Dropout regularization을 추가합니다.

Dropout 적용 확률을 argument로 사용합니다.

<br>

### 중요 참고사항

위의 네트워크 정의를 보면, 첫 번째 Dense layer에만 input_shape라는 argument가 있습니다.

tf.keras.models.Sequential 모듈을 이용하여 네트워크를 정의할 때,

첫 번째 layer에는 input_shape를 지정해 주어야 합니다.

우리가 사용하는 데이터의 shape는 (4,) 입니다.

(위에서 확인한 input shape에서 데이터 개수 부분만 제외)

## 모델 구조 확인

summary() 함수를 이용해 모델 구조를 확인해 볼 수 있습니다.

In [ ]:
model_1.summary()
model_2.summary()

## 모델 학습을 위한 설정

모델 구조를 정의한 후,

model.compile()을 통해 학습 관련 설정을 지정하고 모델을 컴파일합니다.

<br>

Optimizer는 'adam', 'sgd', 'rmsprop' 등을 선택할 수 있습니다.

실습이나 시험에서는 빠른 수렴을 위해 adam을 권장합니다. (실제 업무에 적용시에는 다를 수 있습니다.)

<br>

Loss는 매우 다양한 선택지가 있습니다.

이진 분류 문제에서는 'binary_crossentropy' 를 사용합니다.

Multi-class 분류 문제에서는 'categorical_crossentropy' 또는 'sparse_categorical_crossentropy'를 사용합니다.

만약 Label이 1, 2 등 하나의 정수 형태이면 'sparse_categorical_crossentropy'를 사용합니다.

Label이 [0, 0, 1]와 같이 one-hot encoding된 형태이면 'categorical_crossentropy'를 사용합니다.

위에서 확인했듯이, 우리의 label은 정수 형태이므로 'sparse_categorical_crossentropy'를 사용합니다.

<br>

평가 metric으로는 accuracy를 사용합니다.

In [ ]:
model_1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## 모델 학습

위의 compile을 마쳤다면, 실제로 모델을 학습하고 평가합니다.

fit 함수에 학습 데이터와 label을 전달하고, 학습할 epoch 수를 지정하면 학습이 진행됩니다.

In [ ]:
model_1.fit(x_train, y_train, epochs=30, batch_size=8)

## 모델 평가

evaluate 함수에 평가용 데이터와 label을 전달하여 평가를 진행합니다.

Loss와 accuracy가 표시됩니다.

In [ ]:
model_1.evaluate(x_test,  y_test)